# Evaluation

- **Comments**: 60  
- **Retrieved Claims**: 321  
- **Valid Claims**: 310/321  
- **Czech Claims from Valid**: 79/310  
- **Checkworthy**: 208/310  

In [33]:
import json
import pandas as pd
import numpy as np

In [34]:
import sentencepiece as SentencePiece
from transformers import MarianMTModel, MarianTokenizer
# Load the model and tokenizer for English to Czech
model_name = "Helsinki-NLP/opus-mt-en-cs"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [35]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
detection_model = fasttext.load_model(model_path)


In [36]:
detection_model.predict("Hello, world!")

(('__label__eng_Latn',), array([0.61224753]))

In [37]:
#load data for evaluation (loki generated atomic claims)
with open('data/atomic_claims_comments.json', 'r') as file:
    claims = json.load(file)

# Convert the JSON data to a DataFrame
df = pd.DataFrame(claims)
df = df.explode('claims') 
#normalize json data in claims but keep the source column
new_df = pd.json_normalize(df['claims'])
new_df['source'] = df['source'].values
df = new_df

#drop na rows and id column
df.dropna(axis=0, inplace=True)
df.drop(columns=['id'], inplace=True)
print(df.info())

#for every claim check if in english or not
df['lang'] = df['claim'].apply(lambda x: detection_model.predict(x)[0][0])
df['is_czech'] = df['lang'].apply(lambda x: 1 if (x == '__label__ces_Latn') or (x == '__label__slk_Latn')  else 0)
print(f"Number of claims in Czech: {df['is_czech'].sum()}")
    




<class 'pandas.core.frame.DataFrame'>
Index: 310 entries, 0 to 320
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   claim               310 non-null    object 
 1   checkworthy         310 non-null    object 
 2   checkworthy_reason  310 non-null    object 
 3   origin_text         310 non-null    object 
 4   start               310 non-null    float64
 5   end                 310 non-null    float64
 6   source              310 non-null    object 
dtypes: float64(2), object(5)
memory usage: 19.4+ KB
None
Number of claims in Czech: 79


In [38]:
#translate function
def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

df['original_claim'] = df['claim']
#translate only if the claim is in english
df['claim'] = df.apply(
    lambda row: translate(row['claim']) if row['is_czech'] == 0 else row['claim'], axis=1
)




In [39]:
#checkworthy claims in the dataset-where checkworthy reason starts with Yes
df['checkworthy'] = df['checkworthy_reason'].apply(lambda x: 1 if x.startswith('Yes') else 0)
print(f"Number of checkworthy claims: {df['checkworthy'].sum()}")

#filter out only the rows that are checkworthy 1
filtered_df = df.where(df['checkworthy'] == 1).dropna()

#group by source 
#group by unique comment and comment_id
#df.drop(columns=['checkworthy', 'lang', 'is_czech', 'original_claim'], inplace=True)
df_grouped = filtered_df.groupby(['source']).agg({
    'claim': lambda x: '; '.join(x.astype(str)),
    'checkworthy_reason': lambda x: '; '.join(x.astype(str)),
    'origin_text': lambda x: '; '.join(x.astype(str)),
    'start': lambda x: '; '.join(x.astype(str)),  
    'end': lambda x: '; '.join(x.astype(str))  
}).reset_index()
# Split the joined strings back into lists
df_grouped['claim'] = df_grouped['claim'].apply(lambda x: x.split('; '))
df_grouped['checkworthy_reason'] = df_grouped['checkworthy_reason'].apply(lambda x: x.split('; '))
df_grouped['origin_text'] = df_grouped['origin_text'].apply(lambda x: x.split('; '))
df_grouped['start'] = df_grouped['start'].apply(lambda x: x.split('; '))
df_grouped['end'] = df_grouped['end'].apply(lambda x: x.split('; '))

Number of checkworthy claims: 208


In [40]:

with open('data/atomic_claims_comments_translated.json', 'w') as file:
    json.dump(df_grouped.to_dict(orient='records'), file, ensure_ascii=True, indent=4)
df_grouped.head()

,source,claim,checkworthy_reason,origin_text,start,end
0,"... pokud chci mit data v bezpeci, tak jedine ...",[Data lze udržet v bezpečí prostřednictvím šif...,[Yes (This is a factual statement about a capa...,"[... pokud chci mit data v bezpeci, tak jedine...","[0.0, 63.0, 75.0]","[62.0, 75.0, 75.0]"
1,"A myslíte, že v Linuxu není jen na okrasu? Jak...",[Fyzický přístup k počítači umožňuje nastavení...,[Yes (This statement makes a verifiable techni...,"[Jakmile máte fyzický přístup k počítači, tak ...","[43.0, 122.0, 123.0, 149.0]","[122.0, 122.0, 149.0, 149.0]"
2,A peníze na ty dotace si tisknou? Dotace na el...,[Peníze na dotace se tisknou.],[Yes (This statement makes a factual claim abo...,[A peníze na ty dotace si tisknou?],[0.0],[33.0]
3,"A zapomněli jste na variantu, že je disk šifro...",[Jednoduché zotavení není možné s Bitlocker ši...,[Yes (This statement makes a technical claim a...,"[To už takhle jednoduše nejde., Proto je ve vě...","[82.0, 112.0]","[111.0, 177.0]"
4,"Ale kecy, EURO 6 platí od 1.1.2015, to znamená...",[Emisní norma EURO 6 je platná od 1. ledna 201...,"[Yes (This statement contains specific, verifi...","[Ale kecy, EURO 6 platí od 1.1.2015,, to zname...","[0.0, 36.0]","[35.0, 88.0]"
